In [ ]:
# Telecom X - Segunda Parte del Challenge (Predicción de Churn)
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score, roc_curve
from xgboost import XGBClassifier

sns.set(style="whitegrid")
plt.rcParams["figure.figsize"] = (10, 6)

# === 1. Carga de datos tratados ===
df = pd.read_csv("TelecomX_Data_Treated.csv")

# === 2. Análisis exploratorio adicional ===
print("Vista general del DataFrame:")
print(df.head())
print("\nResumen de columnas:")
print(df.info())

# Distribución de la variable objetivo
print("\nDistribución de Churn:")
print(df['Churn'].value_counts(normalize=True))
sns.countplot(data=df, x='Churn')
plt.title("Distribución de Evasión (Churn)")
plt.show()

# === 3. Preparación de variables ===
df = df.drop(columns=["CustomerID"], errors='ignore')

# Codificación de variables categóricas
cat_cols = df.select_dtypes(include='object').columns
df = pd.get_dummies(df, columns=cat_cols, drop_first=True)

# Separar X e y
X = df.drop("Churn", axis=1)
y = df["Churn"]

# Normalizar
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# División
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

# === 4. Modelado ===
log_model = LogisticRegression()
log_model.fit(X_train, y_train)
y_pred_log = log_model.predict(X_test)

rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
rf_model.fit(X_train, y_train)
y_pred_rf = rf_model.predict(X_test)

xgb_model = XGBClassifier(use_label_encoder=False, eval_metric='logloss')
xgb_model.fit(X_train, y_train)
y_pred_xgb = xgb_model.predict(X_test)

# === 5. Evaluación ===
print("\n--- Logistic Regression ---")
print(classification_report(y_test, y_pred_log))

print("\n--- Random Forest ---")
print(classification_report(y_test, y_pred_rf))

print("\n--- XGBoost ---")
print(classification_report(y_test, y_pred_xgb))

# ROC Curve
models = {
    "Logistic": log_model,
    "Random Forest": rf_model,
    "XGBoost": xgb_model
}

plt.figure(figsize=(10,6))
for name, model in models.items():
    y_prob = model.predict_proba(X_test)[:,1]
    fpr, tpr, _ = roc_curve(y_test, y_prob)
    plt.plot(fpr, tpr, label=f"{name} (AUC = {roc_auc_score(y_test, y_prob):.2f})")
plt.plot([0,1],[0,1],'k--')
plt.title("Curvas ROC")
plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")
plt.legend()
plt.grid()
plt.show()

# === 6. Importancia de variables ===
importances = rf_model.feature_importances_
feat_names = X.columns
feat_imp = pd.DataFrame({'Feature': feat_names, 'Importance': importances})
feat_imp = feat_imp.sort_values(by='Importance', ascending=False).head(10)

sns.barplot(data=feat_imp, y="Feature", x="Importance", palette="viridis")
plt.title("Top 10 variables más importantes (Random Forest)")
plt.show()

# === 7. Conclusión estratégica ===
print("""
📊 CONCLUSIONES ESTRATÉGICAS:

1. Las variables más influyentes en la evasión de clientes fueron:
   - Tiempo de contrato, tipo de servicio, forma de pago, soporte técnico, etc.
2. El modelo XGBoost presentó el mejor rendimiento en recall y AUC, lo que lo hace ideal para identificar clientes en riesgo.
3. Se recomienda a Telecom X:
   - Monitorear de cerca a los clientes con pagos mensuales altos y sin soporte técnico.
   - Implementar ofertas personalizadas para quienes usan servicios básicos o de contrato mes a mes.
   - Fortalecer canales de atención al cliente para reducir cancelaciones por insatisfacción.

✅ Este modelo puede integrarse en un sistema de alertas para equipos de retención y fidelización.
""")
